# Configuração e Carregamento de Múltiplos DataFrames

In [ ]:
!pip install pyspark # nstala o PySpark no ambiente Colab.

In [ ]:
import pyspark # importa a biblioteca PySpark.
from pyspark.sql import SparkSession # Importa a classe SparkSession, usada para iniciar a sessão Spark.

In [ ]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém uma instância do SparkSession

In [5]:
# Estas linhas seguintes carregam sete arquivos CSV de um caminho no Google Drive. Cada arquivo é lido, transformado em um DataFrame
# usando a primeira linha como cabeçalho (header=True) e inferindo os tipos de dados (inferSchema=True)
produtos =  spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True)
vendedores =  spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/vendedores.csv", header=True, inferSchema=True)
clientes =  spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True)
itens_pedido =  spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido =  spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pagamentos_pedido.csv", header=True, inferSchema=True)
avaliacoes_pedido =  spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True)
pedidos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pedidos.csv", header=True, inferSchema=True)

# Inspeção Inicial dos DataFrames

In [6]:
print('dataframe produto: \n', produtos.show(n=5, truncate=False))# Exibe as 5 primeiras linhas do DataFrame produtos. O argumento truncate=False garante que o conteúdo das colunas longas seja exibido na íntegra.
print('dataframe vendedores: \n', vendedores.show(5)) # Exibe as 5 primeiras linhas do DataFrame vendedores
print('dataframe clientes: \n', clientes.show(5)) # Exibe as 5 primeiras linhas do DataFrame clientes
print('dataframe itens_pedido: \n', itens_pedido.show(5) ) # Exibe as 5 primeiras linhas do DataFrame itens_pedido
print('dataframe pagamentos_pedido: \n', pagamentos_pedido.show(5) ) # Exibe as 5 primeiras linhas do DataFrame pagamentos_pedido
print('dataframe  avaliacoes_pedido: \n', avaliacoes_pedido.show(5)) # Exibe as 5 primeiras linhas do DataFrame avaliacoes_pedido
print('dataframe pedidos: \n', pedidos.show(5)) # Exibe as 5 primeiras linhas do DataFrame pedidos

+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|id_produto                      |categoria_produto    |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff4541ed26657ea517e5|perfumaria           |40                  |287                      |1                       |225           |16                    |10               |14                |
|3aa071139cb16b67ca9e5dea641aaa2f|artes                |44                  |276                      |1                       |1000          |30                    |18               |

# Acessando Colunas (Diferentes Sintaxes)

In [7]:
# acessar colunas
from pyspark.sql.functions import col # Importa a função col, que é a forma recomendada de referenciar colunas em operações do PySpark, especialmente dentro de transformações.

clientes.select("id_cliente").show(n=1, truncate=False) # Seleciona a coluna id_cliente usando o nome da coluna como string (o mais simples)
clientes.select(col('id_cliente')).show(1) # Seleciona a coluna usando a função col(). É a melhor prática, pois é mais seguro e flexível em expressões.
clientes.select(clientes['id_cliente']).show(1) # Seleciona a coluna usando a notação de dicionário no próprio DataFrame
clientes.select(clientes.id_cliente).show(1) # Seleciona a coluna usando a notação de objeto (DataFrame.coluna).


+--------------------------------+
|id_cliente                      |
+--------------------------------+
|06b8999e2fba1a1fbc88172c00ba8bc7|
+--------------------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row



# Limpeza e Tratamento de Nulos (na.fill e na.drop)

In [8]:
# Cria um novo DataFrame (produtos_tratar_categ_nulos) onde todos os valores nulos (NaN/Null) na coluna categoria_produto são substituídos pela string 'não especificado'.
produtos_tratar_categ_nulos = produtos.na.fill({'categoria_produto': 'não especificado'})

# Filtra o DataFrame tratado para contar quantas linhas agora têm o valor 'não especificado', verificando a eficácia do preenchimento.
produtos_tratar_categ_nulos.filter(col('categoria_produto') == 'não especificado').count()

610

In [9]:
print('total de pedidos: ', pedidos.count()) # Exibe o número total de registros (linhas) no DataFrame original pedidos

pedidos_unicos = pedidos.dropDuplicates() # Cria um novo DataFrame (pedidos_unicos) removendo todas as linhas que são exatamente iguais em todas as colunas.
print('total de pedidos unicos: ', pedidos_unicos.count()) # Exibe a contagem de linhas após a remoção de duplicatas.

pedidos_remocao_nulos = pedidos_unicos.na.drop() # Cria um novo DataFrame removendo todas as linhas que contêm pelo menos um valor nulo em qualquer coluna.
print('total de pedidos após limpeza de valores nulos: ', pedidos_remocao_nulos.count()) # Exibe a contagem após a remoção de nulos em todas as colunas.

pedidos_remocao_nulos_id = pedidos_unicos.na.drop(subset=['id_cliente', 'id_pedido']) # Cria um novo DataFrame removendo as linhas que contêm valores nulos apenas nas colunas especificadas (id_cliente e id_pedido). Outras colunas com nulos são ignoradas.
print('total de pedidos após limpeza de id nulos: ', pedidos_remocao_nulos_id.count()) # Exibe a contagem após a remoção de nulos em colunas específicas.

total de pedidos:  99441
total de pedidos unicos:  99441
total de pedidos após limpeza de valores nulos:  96461
total de pedidos após limpeza de id nulos:  99441


In [10]:
colunas = ["peso_produto_g", "comprimento_produto_cm", "altura_produto_cm", "largura_produto_cm"] # Define uma lista das colunas relacionadas a medidas físicas dos produtos.

for coluna in colunas:
  produtos = produtos.na.fill({coluna: 0}) # Itera sobre a lista colunas e, para cada coluna, preenche os valores nulos com zero (0)

# Salvamento dos Dados Limpos

In [11]:
# Salva o DataFrame produtos (agora com nulos preenchidos) em um novo local no Google Drive. Usa o modo overwrite e inclui o cabeçalho no arquivo de saída.
produtos.write.mode("overwrite").option("header","true").csv("/content/drive/MyDrive/Material de apoio - M27/output/produtos_tratados.csv")

In [12]:
# Lê o arquivo CSV recém-salvo de volta e exibe as 5 primeiras linhas para confirmar que a operação de escrita foi bem-sucedida.
spark.read.option('header','true').csv("/content/drive/MyDrive/Material de apoio - M27/output/produtos_tratados.csv").show(5)

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  